In [2]:
# Importing the required libraries
from random import random
from typing import Callable, List, Tuple
from random import choices, randint, randrange
from functools import partial
import time

# Defining each variable and function type
Chromosome = List[int]
Population = List[Chromosome]
FitnessFunc = Callable[[Chromosome], float]
PopulationFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Chromosome, Chromosome]]
CrossoverFunc = Callable[[Chromosome, Chromosome], Tuple[Chromosome, Chromosome]]
MutationFunc = Callable[[Chromosome], Chromosome]

# A general purpose thing object to store each item
class Food:
    def __init__(set, food, value, calorie):
        set.food = food
        set.value = value
        set.calorie = calorie

things = []

# Defining all of the above mentioned functions

def generate_chromosome(length: int) -> Chromosome:
    return choices([0, 1], k=length)

def generate_population(size: int, chromosome_length: int) -> Population:
    return [generate_chromosome(chromosome_length) for _ in range(size)]

def fitness(chromosome: Chromosome, things: List[Food], calorie_limit: float) -> float:
    if len(chromosome) != len(things):
        raise ValueError("Chromosome and Food must be the same length")

    calories = 0
    value = 0
    for i, thing in enumerate(things):
        if chromosome[i] == 1:
            calories += thing.calorie
            value += thing.value

            if calories > calorie_limit:
                return 0

    return value

def selection(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        calorie=[fitness_func(chromosome) for chromosome in population],
        k=2
    )

def crossover(a: Chromosome, b: Chromosome) -> Tuple[Chromosome, Chromosome]:
    if len(a) != len(b):
        raise ValueError("First and second chromosome need to be the same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length-1)
    return a[:p] + b[p:], b[:p] + a[p:]

def mutation(chromosome: Chromosome, num: int = 1, probability: float = 0.5) -> Chromosome:
    for _ in range(num):
        index = randrange(len(chromosome))
        chromosome[index] = chromosome[index] if random() > probability else abs(chromosome[index] - 1)
    return chromosome

def evolution(
    populate_func: PopulationFunc,
    fitness_func: FitnessFunc,
    selection_func: SelectionFunc = selection,
    crossover_func: CrossoverFunc = crossover,
    mutation_func: MutationFunc = mutation,
    generation_limit: int = 100
) -> Tuple[Population, int]:

    population = populate_func()

    for i in range(generation_limit):
        population = sorted(
            population,
            key=lambda chromosome: fitness_func(chromosome),
            reverse=True
        )

        next_generation = population[:2]

        for j in range(int(len(population)/2)-1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    population = sorted(
        population,
        key=lambda chromosome: fitness_func(chromosome),
        reverse=True
    )

    return population, i

def chromosome(chromosome: Chromosome, things: List[Food]) -> List[str]:
    result = []
    for i, thing in enumerate(things):
        if chromosome[i] == 1:
            result += [thing.food]

    return result

# Taking input of the list of items
print("Enter the number of food you want: ")
num_food = int(input())

for i in range(num_food):
    print(f"Enter the food {i+1}: ")
    item_food = input()
    print(f"Enter the value of food {i+1}: ")
    item_value = float(input())
    print(f"Enter the calorie amount of food {i+1}: ")
    item_calorie = float(input())
    print("\n")
    things.append(Food(item_food, item_value, item_calorie))

# Defining the weight and fitness limits
print("Enter the calorie limit per day: ")
inp_calorie_limit = float(input())

start = time.time()
population, generations = evolution(
    populate_func=partial(
        generate_population, size=10, chromosome_length=len(things)
    ),
    fitness_func=partial(
        fitness, things=things, calorie_limit=inp_calorie_limit
    ),
    generation_limit=503
)
end = time.time()

print(f"\nNumber of generations: {generations}")
print(f"Time: {end - start}s")
print(f"Best food to consume: {chromosome(population[0], things)}")
print(f"Value: {fitness(population[0], things, inp_calorie_limit)}")

Enter the number of food you want: 
10
Enter the food 1: 
beef
Enter the value of food 1: 
3
Enter the calorie amount of food 1: 
250


Enter the food 2: 
chicken
Enter the value of food 2: 
6
Enter the calorie amount of food 2: 
165


Enter the food 3: 
rice
Enter the value of food 3: 
4
Enter the calorie amount of food 3: 
136


Enter the food 4: 
bread
Enter the value of food 4: 
5
Enter the calorie amount of food 4: 
82


Enter the food 5: 
banana
Enter the value of food 5: 
7
Enter the calorie amount of food 5: 
110


Enter the food 6: 
soda
Enter the value of food 6: 
2
Enter the calorie amount of food 6: 
140


Enter the food 7: 
broccoli
Enter the value of food 7: 
10
Enter the calorie amount of food 7: 
30


Enter the food 8: 
spinach
Enter the value of food 8: 
8
Enter the calorie amount of food 8: 
23


Enter the food 9: 
burger
Enter the value of food 9: 
1
Enter the calorie amount of food 9: 
300


Enter the food 10: 
9
Enter the value of food 10: 
9
Enter the calorie amou

TypeError: ignored